In [ ]:
%pip install moviepy

In [ ]:
import os
import numpy as np
from moviepy.editor import VideoFileClip

# Clip extraction

*Given a start and end time (i.e. for a piece, movement - the smallest 'continuous' segment of music)* \
*Window = 10 sec*

Method 1. Sliding window, no overlap \
Method 2. Sliding window, overlap (___ sec) \
Method 3. Random sampling (____ clips per ____ sec segment)

## Notes

1. Videos are in 720p if possible, otherwise whatever's the highest i can get

In [ ]:
filename = ""
vid = VideoFileClip(filename)

In [ ]:
root_dir = "data"
frame_dir = os.path.join(root_dir, "frames")
audio_dir = os.path.join(root_dir, "audio")

if not os.path.exists(frame_dir):
    os.makedirs(frame_dir)

if not os.path.exists(audio_dir):
    os.makedirs(audio_dir)

In [ ]:
video_fps = 30

## Segments so I don't forget

Fast:
1. La campanella: 0:05 to 4:25
2. scarbo: 0:10 to 8:24
3. 

Slow:
1. tristesse: 0:05 to 4:10
2. 

bit of both:
1. chopin ballade: 0:05 to 9:25
2. hammerklavier (full): 0:00 to 10:32, 10:39 to 13:09, 13:15 to 28:45, 29:00 to 39:29
3. goldberg variations (full): 0:05 to 1:23:22
4. 

AIM FOR 30 HOURS (a bit over 10k samples)

In [ ]:
# Define segment of video to extract from
start_sec = 0
end_sec = 0

dur = 10

## Method 1

In [ ]:
clip_counter = 0
for i in range(start_sec, end_sec, dur):
    
    if (end_sec - i) < dur:
        break
    
    clip_vid_dir = os.path.join(frame_dir, str(clip_counter))
    if os.path.exists(clip_vid_dir):
        os.makedirs(clip_vid_dir)

    clip = vid.subclip(i, i+dur)

    clip_audio = clip.audio

    # extract, preprocess, and save frames
    # TODO: optical flow, contouring?
    frame_counter = 0
    for frame in clip.iter_frames(fps=video_fps):
        # frame is a H x W x N (N = 3 for RGB) np.array
        frame_path = os.path.join(clip_vid_dir, str(frame_counter) + ".npy")
        np.save(frame_path, frame)
        frame_counter += 1

    clip_counter += 1